In [58]:
import os
from pathlib import Path
import numpy as np
import scipy
Downloads2 = str(os.path.join(Path.home(), 'downloads','reactions.txt'))
reactions = np.genfromtxt(fname=Downloads2, delimiter='/n', dtype='unicode')
print(reactions)

from decimal import Decimal

#--IMPORTED DATA FROM FILE--#

def stoichiometric_coefficient(molecule):
    position = 0
    coefficient = ''
    while position < len(molecule):
        if molecule[position].isdigit():
            coefficient += molecule[position]
        else:
            break
        position += 1
    if coefficient == '':
        just_molecule = molecule
        return (1,just_molecule)
    else:
        just_molecule = molecule[len(coefficient):]
        return (int(coefficient),just_molecule)
def count_atoms(molecule):
    charge = 0
    poscharge = molecule.find('+')
    negcharge = molecule.find('-')
    if poscharge != -1:
        charge = molecule.split('+')[1]
        if charge == '':
            charge = 1
        else:
            charge = int(charge)
        molecule = molecule.split('+')[0]
    if negcharge != -1:
        charge = molecule.split('-')[1]
        if charge == '':
            charge = -1
        else:
            charge = -int(charge)
        molecule = molecule.split('-')[0]
    position = 0
    atoms = {}
    if charge != 0:
        atoms['e'] = charge
    current_atom = ''
    current_number = ''
    for x in molecule:
        if x.isupper():
            if current_atom != '':
                if current_number == '': 
                    if current_atom in atoms.keys():
                        atoms[current_atom] += 1
                    else:
                        atoms[current_atom] = 1
                else:
                    if current_atom in atoms.keys():
                        atoms[current_atom] += int(current_number)
                    else:
                        atoms[current_atom] = int(current_number)
                current_atom = ''
                current_number = ''
            current_atom += x
        elif x.islower():
            current_atom += x
        else:
            current_number += x
    if current_atom != '':
        if current_number == '':
            if current_atom in atoms.keys():
                atoms[current_atom] += 1
            else:
                atoms[current_atom] = 1
        else:
            if current_atom in atoms.keys():
                atoms[current_atom] += int(current_number)
            else:
                atoms[current_atom] = int(current_number)
    return atoms

#--DEFINED IMPORTANT VARIABLES --#
#--BEGIN MAJOR FOR LOOP--#

for x in reactions:
    all_reactants = []
    all_products = []
    reactants_together = x.split(" -> ")[0]
    products_together = x.split(" -> ")[1]
    reactants = reactants_together.split(" + ")
    products = products_together.split(" + ")

    #--SPLITS EACH REACTION INPUT--#
    
    allreactants = {}
    allproducts = {}
    for reactant in reactants:
        (n,mol) = stoichiometric_coefficient(reactant)
        atoms = count_atoms(mol)
        for atom in atoms.keys():
            if atom in allreactants.keys():
                allreactants[atom] += n * atoms[atom]
            else:
                allreactants[atom] = n * atoms[atom]
    for product in products:
        (n,mol) = stoichiometric_coefficient(product)
        atoms = count_atoms(mol)
        for atom in atoms.keys():
            if atom in allproducts.keys():
                allproducts[atom] += n * atoms[atom]
            else:
                allproducts[atom] = n * atoms[atom]
    atoms = sorted(allreactants.keys())
    nrows = len(atoms)
    ncolumns = len(reactants) + len(products) - 1
    mat = np.zeros((nrows,ncolumns))
    column = 0
    for compound in reactants[1:]:
        (n,mol) = stoichiometric_coefficient(compound)
        atoms_here = count_atoms(mol)
        for atomtype in range(len(atoms)):
            if atoms[atomtype] in atoms_here.keys():
                mat[atomtype,column] = -atoms_here[atoms[atomtype]]
        column += 1
    for compound in products:
        (n,mol) = stoichiometric_coefficient(compound)
        atoms_here = count_atoms(mol)
        for atomtype in range(len(atoms)):
            if atoms[atomtype] in atoms_here.keys():
                mat[atomtype,column] = atoms_here[atoms[atomtype]]
        column += 1
    rhs = np.zeros(nrows)
    (n,mol) = stoichiometric_coefficient(reactants[0])
    atoms_here = count_atoms(mol)
    for atomtype in range(len(atoms)):
        if atoms[atomtype] in atoms_here.keys():
            rhs[atomtype] = atoms_here[atoms[atomtype]]
    res = scipy.linalg.lstsq(mat, rhs)
    new_coefficients = res[0]
    new_coefficientslist = new_coefficients.tolist()
    new_cl = new_coefficientslist[:]
    new_cl.insert(0, 1.0)
    
    multiplier = 1
    while True:
        problem = False
        for x in new_cl:
            if abs(multiplier*x - round(multiplier*x))>0.001:
                problem = True
                break
        if not(problem):
            break
        multiplier += 1
    balanced_coeff = []    
    balanced = []
    for x in new_cl:
        balanced = round(x*multiplier)
        balanced_coeff.append(balanced)
                
    finalstring = ''
    position = 0
    
    for x in reactants:
        if balanced_coeff[position] == 1:
            finalstring += x
        if balanced_coeff[position] != 1:
            finalstring += str(balanced_coeff[position]) + x
        position += 1
        if position < len(reactants):
            finalstring += ' + '
        
    finalstring += ' -> '
    
    for x in products:
        if balanced_coeff[position] == 1:
            finalstring += x
        if balanced_coeff[position] != 1:
            finalstring += str(balanced_coeff[position]) + x
        position += 1
        if position < len(products)+len(reactants):
            finalstring += ' + '
            
            
    print(finalstring)

['C2H5OH + O2 -> CO2 + H2O' 'CrO4-2 + H+ -> Cr2O7-2 + H2O'
 'KMnO4 + KI + HCl -> KCl + MnCl2 + I2 + H2O'
 'MnO4- + I- + H+ -> Mn+2 + I2 + H2O'
 'HgS2C2N2 + O2 -> HgS + CO2 + SO2 + C3N4']
C2H5OH + 3O2 -> 2CO2 + 3H2O
2CrO4-2 + 2H+ -> Cr2O7-2 + H2O
2KMnO4 + 10KI + 16HCl -> 12KCl + 2MnCl2 + 5I2 + 8H2O
2MnO4- + 10I- + 16H+ -> 2Mn+2 + 5I2 + 8H2O
2HgS2C2N2 + 3O2 -> 2HgS + CO2 + 2SO2 + C3N4
